In [36]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

pd.options.display.max_columns = 999

import shapely as sp
import shapely.geometry as sgeom
import cartopy.io.shapereader as shpreader

from math import *


In [2]:
import folium
#import branca.colormap as cm
#import json

In [3]:
def read_file(fname):

    reader = shpreader.Reader(fname)
    geoms = list(reader.geometries())
    records = list(reader.records())
    gdf = gpd.read_file(fname)

    df = pd.DataFrame([x.attributes for x in records])
    return gdf

In [4]:
tsp_gdf = read_file('data/townships/myanmar_township_boundaries.shp')
sar_gdf = read_file('data/townships/myanmar_self_administered_regions_boundaries.shp')

In [5]:
sar_gdf.head()

,OBJECTID,ST,ST_PCODE,DT,DT_PCODE,TS,TS_MYA,TS_PCODE,ST_2,LABEL2,S_ADMIN,ST_RG,AREA,geometry
0,203.0,Shan (South),MMR014,Taunggyi,MMR014D001,Pindaya,áááºá¸áá,MMR014006,Shan State (South),Pindaya\n77769,Danu,State,629.024963,"POLYGON ((96.77309906800012 21.04325973900007,..."
1,240.0,Shan (South),MMR014,Taunggyi,MMR014D001,Ywangan,áá½á¬áá¶,MMR014007,Shan State (South),Ywangan\n76933,Danu,State,2984.376944,"POLYGON ((96.7848338460001 21.73454093300002, ..."
2,106.0,Shan (South),MMR014,Taunggyi,MMR014D001,Pinlaung,áááºáá±á¬ááºá¸,MMR014009,Shan State (South),Pinlaung\n162537,Pa-O,State,3396.963489,"POLYGON ((96.49517839500007 20.4798822220001, ..."
3,134.0,Sagaing,MMR005,Hkamti,MMR005D008,Nanyun,áááºá¸áá½ááºá¸,MMR005037,Sagaing Region,Nanyun\n78605,Naga,Region,6613.151755,"POLYGON ((95.90216654300004 26.48919462400008,..."
4,77.0,Sagaing,MMR005,Hkamti,MMR005D008,Lay Shi,áá±áá¾á®á¸,MMR005035,Sagaing Region,None,Naga,Region,2792.837366,"POLYGON ((94.67146862700005 24.87438462500003,..."


In [6]:
all_gdfs = [tsp_gdf,sar_gdf]

In [7]:
def get_index_of_tsp_location(lat,lon,gdfs):
    gdfs_list = []
    if isinstance(gdfs, list):
        gdfs_list = gdfs
    else:
        gdfs_list.append(gdfs)
    for g in gdfs_list:
        #display(g.head())
        for i, t in g.iterrows():
            g_type = t.geometry.geom_type
            if (g_type == 'Polygon'):
                polygon = sgeom.polygon.Polygon(t.geometry)
            elif (g_type == 'MultiPolygon'):
                polygon = sgeom.multipolygon.MultiPolygon(t.geometry)
            if polygon.contains(sgeom.Point(lon, lat)):
                return pd.DataFrame(t).T
    return pd.DataFrame(columns=gdfs[0].columns)

In [8]:
# testing a point in Hpakant

get_index_of_tsp_location(25.6787,96.3858,all_gdfs)

,OBJECTID,ST,ST_PCODE,DT,DT_PCODE,TS,TS_PCODE,ST_2,LABEL2,SELF_ADMIN,ST_RG,T_NAME_WIN,T_NAME_M3,AREA,geometry
0,250,Kachin,MMR001,Mohnyin,MMR001D002,Hpakant,MMR001009,Kachin State,Hpakant\n169795,None,State,zm;uefU,áá¬á¸ááá·áº,5761.3,"POLYGON ((96.15952987500003 26.11798109600001,..."


In [9]:
# testing a point not in Myanmar, should return empty

get_index_of_tsp_location(0,1,all_gdfs)


,OBJECTID,ST,ST_PCODE,DT,DT_PCODE,TS,TS_PCODE,ST_2,LABEL2,SELF_ADMIN,ST_RG,T_NAME_WIN,T_NAME_M3,AREA,geometry


In [10]:
# testing a point not in Hopang, Wa SAR

get_index_of_tsp_location(23.4254,98.7556,all_gdfs)


,OBJECTID,ST,ST_PCODE,DT,DT_PCODE,TS,TS_PCODE,ST_2,LABEL2,SELF_ADMIN,ST_RG,T_NAME_WIN,T_NAME_M3,AREA,geometry
315,224,Shan (North),MMR015,Hopang,MMR015D006,Hopang,MMR015021,Shan State (North),Hopang\n24637,Wa,State,[dkyef,áá­á¯áááº,1332.38,"POLYGON ((98.51675274000003 22.95548887200003,..."


In [11]:
def viz_point(lat,lon,gdfs):
    location_result = get_index_of_tsp_location(25.6787,96.3858,gdfs)
    #location_result = get_attributes_of_tsp_location(16.8728,96.0685,geoms,df)
    map = folium.Map(location=[lat,lon], zoom_start=12)
    
    html="""
    <strong>Township: </strong>{tsp}<br/>
    <strong>State/Region: </strong>{st}<br/>
    <strong>Pcode: </strong>{pcode}
    """
    table = folium.Html(html.format(
            tsp = location_result['TS'],
            st = location_result['ST'],
            pcode = location_result['TS_PCODE']
        ), script=True)
    popup = folium.Popup(table)
    
    folium.Marker([lat, lon], popup=popup).add_to(map)
    return map

In [12]:
viz_point(16.8728,96.0685,all_gdfs)

## Read from csv and add columns with township, state and Pcode

In [13]:
df = pd.read_csv('data/Coordinates.csv',sep=';')

In [14]:
df.head(10)

,ID,longitude,latitude,Unnamed: 3
0,1,95.854584,17.497511,NaN
1,2,98.133858,22.036200,NaN
2,3,99.283318,20.381680,NaN
3,4,98.133858,22.036200,NaN
4,5,98.400002,22.483334,NaN
5,6,97.602486,23.714504,NaN
6,7,96.368736,24.767860,NaN
7,8,96.538742,23.649521,NaN
8,9,96.578339,25.703962,NaN
9,10,96.348228,25.620737,NaN


#### Drop extra column from csv file

In [15]:
df.drop(['Unnamed: 3'], axis=1, inplace=True)

#### Apply the get_attributes_of_tsp_location() function to every row in the dataframe and merge it with original dataframe

In [16]:
new_cols = ['ST','TS','TS_PCODE']
new_df = pd.DataFrame(columns=list(df)+new_cols)

for i,row in df.head(30).iterrows():
    lookup = get_index_of_tsp_location(row['latitude'],row['longitude'],all_gdfs)[new_cols]
    new_row = row.to_frame().T.reset_index(drop=True).combine_first(lookup.reset_index(drop=True))
    new_df = pd.concat([new_df,new_row],sort=False)


In [17]:
new_df.head(20)

,ID,longitude,latitude,ST,TS,TS_PCODE
0,1.0,95.854584,17.497511,Yangon,Taikkyi,MMR013005
0,2.0,98.133858,22.036200,Shan (South),Kyethi,MMR014015
0,3.0,99.283318,20.381680,Shan (East),Monghsat,MMR016006
0,4.0,98.133858,22.036200,Shan (South),Kyethi,MMR014015
0,5.0,98.400002,22.483334,Shan (North),Tangyan,MMR015004
0,6.0,97.602486,23.714504,Shan (North),Namhkan,MMR015010
0,7.0,96.368736,24.767860,Kachin,Mohnyin,MMR001007
0,8.0,96.538742,23.649521,Shan (North),Mabein,MMR015018
0,9.0,96.578339,25.703962,Kachin,Hpakant,MMR001009
0,10.0,96.348228,25.620737,Kachin,Hpakant,MMR001009


In [18]:
new_df[new_df.isnull().any(axis=1)]

,ID,longitude,latitude,ST,TS,TS_PCODE
0,11.0,97.734428,23.983349,NaN,NaN,NaN


In [19]:
## Looks like the reason why we are null values for the Pcode lookups is because some of the points are not in Myanmar

null_points = new_df[new_df.isnull().any(axis=1)]

for i,row in null_points.iterrows():
    print(row)
    display(viz_point(row['latitude'],row['longitude'],all_gdfs))

ID                11
longitude    97.7344
latitude     23.9833
ST               NaN
TS               NaN
TS_PCODE         NaN
Name: 0, dtype: object


#### Save output as a csv file

In [20]:
new_df.to_csv('data/Coordinates_with_location.csv',sep=',')

## Test on Hydropower dataset

In [49]:
df = gpd.read_file('data/myanmar_dams.geojson',encoding="ISO-8859-1")

In [50]:
df.head()

,Catchment Annual Rainfall (mm),Total Powerplant Design Discharge (m3/s),Transmission Line Length (km),Transmission Line Voltage level (kV),Turbine Discharge/ Min Flow,Transmission Line Connected to S/S,Full Supply Level (FSL) (m),Powerhouse Type,Dam Type,Reservoir Area (km2),Estimated No. Construction Workers (planned projects),Total Length of Reservoir (km),Reservoir Live Storage (hm3),Reservoir Dead Storage (hm3),Reservoir Retention Period (days),Run-of-River or Storage,Township,Myanmar Selfuse (MW),% use MYA,Export (MW),Height (m),Qmin/ MAF,Crest Length (m),Distance along river between dam and powerhouse (km),Plant Factor,Installed Capacity (MW),Financiers,GIS Rainfall,Max reservoir area (FSL) (km2),Unit Flow (l/s/km2),District,Mean annual inflow (MAF) into reservoir (m3/s),Developers,Sub-Basin (GIS Maps),Status (labelled by IFC),State/Region,Basin,Annual Runoff into Reservoir (hm3),Catchment Area (km2),Sub-Basin (HP Chapter),Mean annual energy (GW),Total storage (FSL) (mil. m3),Notes,River,Function,Country,Location_L,Location_1,Status,Project Name,COD Year,Project ID,geometry
0,NaN,46.0,5.0,132,1.6,Baluchaung 2,867.0,Above ground,CG,0.0,NaN,0.2,2.0,0.0,1.0,RoR,Loikaw,28.0,1.0,0.0,11,0.81,NaN,10.3,0.82,28.0,Sole Investment,1493.0,0.0,4.6,Loikaw,36.5,MoEE,Baluchaung,Built,Kayah,Thanlwin,1151.0,7872.0,Nam Pawn,28,2,http://globalenergyobservatory.org/geoid/41487,Baluchaung,HP,Myanmar,97.287000,19.648000,Complete,Baluchaung 1,1992,DAM0001,POINT (97.28700000000001 19.648)
1,NaN,48.0,0.0,230,1.6,Taungoo and Shwemyo,787.0,Above ground,Weir,0.0,NaN,0.0,0.0,0.0,0.0,RoR,Loikaw,168.0,1.0,0.0,-,0.79,NaN,7.1,0.81,168.0,Sole Investment,1493.0,0.0,4.8,Loikaw,38.0,MoEE,Baluchaung,Built,Kayah,Thanlwin,1198.0,7910.0,Nam Pawn,168,-,This is definitely Baluchaung II. The sequence...,Baluchaung,HP,Myanmar,97.358000,19.557000,Complete,Baluchaung 2,1974,DAM0002,POINT (97.358 19.557)
2,NaN,51.0,5.0,132,1.7,Baluchaung 3,339.0,Above ground,Intake,0.0,NaN,0.0,0.0,0.0,0.0,RoR,Loikaw,52.0,1.0,0.0,-,0.75,NaN,5.1,0.73,52.0,Local BOT,1493.0,NaN,5.0,Loikaw,40.2,Future Energy,Baluchaung,Built,Kayah,Thanlwin,1268.0,8042.0,Nam Pawn,52,-,Not sure if this is a dam - may just be infras...,Baluchaung,HP,Myanmar,97.397000,19.546000,Complete,Baluchaung 3,2014,DAM0003,POINT (97.39700000000001 19.546)
3,1925.0,16.0,32.0,66,3.3,Kalaw,1140.0,Above ground,None,9.7,NaN,NaN,NaN,NaN,NaN,None,Pin Laung,30.0,1.0,0.0,35,0.43,NaN,3.0,0.34,30.0,Local BOT,1493.0,NaN,31.1,Taunggyi,11.1,NeoEnergy Oasis,Baluchaung,Construction,Shan,Thanlwin,349.0,356.0,Nam Pawn,30,None,https://cdm.unfccc.int/filestorage/M/A/B/MABJR...,Baluchaung,HP,Myanmar,96.781918,20.484592,UC,Baluchaung (upper),2019,DAM0004,POINT (96.781918 20.484592)
4,NaN,NaN,NaN,None,NaN,None,NaN,None,Rockfill,19.4,700.0,NaN,278.0,557.0,339.0,S,Kyauk Gyi,160.0,1.0,0.0,80,0.09,NaN,0.0,0.36,160.0,Local BOT,2806.0,NaN,36.5,Taungoo,9.5,Thoolei,Bawgata,LocMoU,Bago,Sittaung,300.0,260.0,Bawgata,160,835,None,Bawgata,HP,Myanmar,96.848496,18.255561,Proposed,Bawgata,2020,DAM0005,POINT (96.848496 18.255561)


In [52]:
new_cols = ['ST','TS','TS_PCODE']
new_df = pd.DataFrame(columns=list(df)+new_cols)

for i,row in df.iterrows():
    lookup = get_index_of_tsp_location(row['Location_1'],row['Location_L'],all_gdfs)[new_cols]
    new_row = row.to_frame().T.reset_index(drop=True).combine_first(lookup.reset_index(drop=True))
    new_df = pd.concat([new_df,new_row],sort=False)

In [53]:
new_df.head()

,Catchment Annual Rainfall (mm),Total Powerplant Design Discharge (m3/s),Transmission Line Length (km),Transmission Line Voltage level (kV),Turbine Discharge/ Min Flow,Transmission Line Connected to S/S,Full Supply Level (FSL) (m),Powerhouse Type,Dam Type,Reservoir Area (km2),Estimated No. Construction Workers (planned projects),Total Length of Reservoir (km),Reservoir Live Storage (hm3),Reservoir Dead Storage (hm3),Reservoir Retention Period (days),Run-of-River or Storage,Township,Myanmar Selfuse (MW),% use MYA,Export (MW),Height (m),Qmin/ MAF,Crest Length (m),Distance along river between dam and powerhouse (km),Plant Factor,Installed Capacity (MW),Financiers,GIS Rainfall,Max reservoir area (FSL) (km2),Unit Flow (l/s/km2),District,Mean annual inflow (MAF) into reservoir (m3/s),Developers,Sub-Basin (GIS Maps),Status (labelled by IFC),State/Region,Basin,Annual Runoff into Reservoir (hm3),Catchment Area (km2),Sub-Basin (HP Chapter),Mean annual energy (GW),Total storage (FSL) (mil. m3),Notes,River,Function,Country,Location_L,Location_1,Status,Project Name,COD Year,Project ID,geometry,ST,TS,TS_PCODE
0,NaN,46.0,5.0,132,1.6,Baluchaung 2,867.0,Above ground,CG,0.0,NaN,0.2,2.0,0.0,1.0,RoR,Loikaw,28.0,1.0,0.0,11,0.81,NaN,10.3,0.82,28.0,Sole Investment,1493.0,0.0,4.6,Loikaw,36.5,MoEE,Baluchaung,Built,Kayah,Thanlwin,1151.0,7872.0,Nam Pawn,28,2,http://globalenergyobservatory.org/geoid/41487,Baluchaung,HP,Myanmar,97.287000,19.648000,Complete,Baluchaung 1,1992,DAM0001,POINT (97.28700000000001 19.648),Kayah,Loikaw,MMR002001
0,NaN,48.0,0.0,230,1.6,Taungoo and Shwemyo,787.0,Above ground,Weir,0.0,NaN,0.0,0.0,0.0,0.0,RoR,Loikaw,168.0,1.0,0.0,-,0.79,NaN,7.1,0.81,168.0,Sole Investment,1493.0,0.0,4.8,Loikaw,38.0,MoEE,Baluchaung,Built,Kayah,Thanlwin,1198.0,7910.0,Nam Pawn,168,-,This is definitely Baluchaung II. The sequence...,Baluchaung,HP,Myanmar,97.358000,19.557000,Complete,Baluchaung 2,1974,DAM0002,POINT (97.358 19.557),Kayah,Loikaw,MMR002001
0,NaN,51.0,5.0,132,1.7,Baluchaung 3,339.0,Above ground,Intake,0.0,NaN,0.0,0.0,0.0,0.0,RoR,Loikaw,52.0,1.0,0.0,-,0.75,NaN,5.1,0.73,52.0,Local BOT,1493.0,NaN,5.0,Loikaw,40.2,Future Energy,Baluchaung,Built,Kayah,Thanlwin,1268.0,8042.0,Nam Pawn,52,-,Not sure if this is a dam - may just be infras...,Baluchaung,HP,Myanmar,97.397000,19.546000,Complete,Baluchaung 3,2014,DAM0003,POINT (97.39700000000001 19.546),Kayah,Loikaw,MMR002001
0,1925.0,16.0,32.0,66,3.3,Kalaw,1140.0,Above ground,None,9.7,NaN,NaN,NaN,NaN,NaN,None,Pin Laung,30.0,1.0,0.0,35,0.43,NaN,3.0,0.34,30.0,Local BOT,1493.0,NaN,31.1,Taunggyi,11.1,NeoEnergy Oasis,Baluchaung,Construction,Shan,Thanlwin,349.0,356.0,Nam Pawn,30,None,https://cdm.unfccc.int/filestorage/M/A/B/MABJR...,Baluchaung,HP,Myanmar,96.781918,20.484592,UC,Baluchaung (upper),2019,DAM0004,POINT (96.781918 20.484592),Shan (South),Pinlaung,MMR014009
0,NaN,NaN,NaN,None,NaN,None,NaN,None,Rockfill,19.4,700.0,NaN,278.0,557.0,339.0,S,Kyauk Gyi,160.0,1.0,0.0,80,0.09,NaN,0.0,0.36,160.0,Local BOT,2806.0,NaN,36.5,Taungoo,9.5,Thoolei,Bawgata,LocMoU,Bago,Sittaung,300.0,260.0,Bawgata,160,835,None,Bawgata,HP,Myanmar,96.848496,18.255561,Proposed,Bawgata,2020,DAM0005,POINT (96.848496 18.255561),Bago (East),Kyaukkyi,MMR007011


In [56]:
new_df

,Catchment Annual Rainfall (mm),Total Powerplant Design Discharge (m3/s),Transmission Line Length (km),Transmission Line Voltage level (kV),Turbine Discharge/ Min Flow,Transmission Line Connected to S/S,Full Supply Level (FSL) (m),Powerhouse Type,Dam Type,Reservoir Area (km2),Estimated No. Construction Workers (planned projects),Total Length of Reservoir (km),Reservoir Live Storage (hm3),Reservoir Dead Storage (hm3),Reservoir Retention Period (days),Run-of-River or Storage,Township,Myanmar Selfuse (MW),% use MYA,Export (MW),Height (m),Qmin/ MAF,Crest Length (m),Distance along river between dam and powerhouse (km),Plant Factor,Installed Capacity (MW),Financiers,GIS Rainfall,Max reservoir area (FSL) (km2),Unit Flow (l/s/km2),District,Mean annual inflow (MAF) into reservoir (m3/s),Developers,Sub-Basin (GIS Maps),Status (labelled by IFC),State/Region,Basin,Annual Runoff into Reservoir (hm3),Catchment Area (km2),Sub-Basin (HP Chapter),Mean annual energy (GW),Total storage (FSL) (mil. m3),Notes,River,Function,Country,Location_L,Location_1,Status,Project Name,COD Year,Project ID,geometry,ST,TS,TS_PCODE
0,NaN,46.0,5.0,132,1.6,Baluchaung 2,867.0,Above ground,CG,0.0,NaN,0.2,2.0,0.0,1.0,RoR,Loikaw,28.0,1.00,0.0,11,0.81,NaN,10.3,0.82,28.0,Sole Investment,1493.0,0.00,4.6,Loikaw,36.5,MoEE,Baluchaung,Built,Kayah,Thanlwin,1151.0,7872.0,Nam Pawn,28,2,http://globalenergyobservatory.org/geoid/41487,Baluchaung,HP,Myanmar,97.287000,19.648000,Complete,Baluchaung 1,1992,DAM0001,POINT (97.28700000000001 19.648),Kayah,Loikaw,MMR002001
0,NaN,48.0,0.0,230,1.6,Taungoo and Shwemyo,787.0,Above ground,Weir,0.0,NaN,0.0,0.0,0.0,0.0,RoR,Loikaw,168.0,1.00,0.0,-,0.79,NaN,7.1,0.81,168.0,Sole Investment,1493.0,0.00,4.8,Loikaw,38.0,MoEE,Baluchaung,Built,Kayah,Thanlwin,1198.0,7910.0,Nam Pawn,168,-,This is definitely Baluchaung II. The sequence...,Baluchaung,HP,Myanmar,97.358000,19.557000,Complete,Baluchaung 2,1974,DAM0002,POINT (97.358 19.557),Kayah,Loikaw,MMR002001
0,NaN,51.0,5.0,132,1.7,Baluchaung 3,339.0,Above ground,Intake,0.0,NaN,0.0,0.0,0.0,0.0,RoR,Loikaw,52.0,1.00,0.0,-,0.75,NaN,5.1,0.73,52.0,Local BOT,1493.0,NaN,5.0,Loikaw,40.2,Future Energy,Baluchaung,Built,Kayah,Thanlwin,1268.0,8042.0,Nam Pawn,52,-,Not sure if this is a dam - may just be infras...,Baluchaung,HP,Myanmar,97.397000,19.546000,Complete,Baluchaung 3,2014,DAM0003,POINT (97.39700000000001 19.546),Kayah,Loikaw,MMR002001
0,1925.0,16.0,32.0,66,3.3,Kalaw,1140.0,Above ground,None,9.7,NaN,NaN,NaN,NaN,NaN,None,Pin Laung,30.0,1.00,0.0,35,0.43,NaN,3.0,0.34,30.0,Local BOT,1493.0,NaN,31.1,Taunggyi,11.1,NeoEnergy Oasis,Baluchaung,Construction,Shan,Thanlwin,349.0,356.0,Nam Pawn,30,None,https://cdm.unfccc.int/filestorage/M/A/B/MABJR...,Baluchaung,HP,Myanmar,96.781918,20.484592,UC,Baluchaung (upper),2019,DAM0004,POINT (96.781918 20.484592),Shan (South),Pinlaung,MMR014009
0,NaN,NaN,NaN,None,NaN,None,NaN,None,Rockfill,19.4,700.0,NaN,278.0,557.0,339.0,S,Kyauk Gyi,160.0,1.00,0.0,80,0.09,NaN,0.0,0.36,160.0,Local BOT,2806.0,NaN,36.5,Taungoo,9.5,Thoolei,Bawgata,LocMoU,Bago,Sittaung,300.0,260.0,Bawgata,160,835,None,Bawgata,HP,Myanmar,96.848496,18.255561,Proposed,Bawgata,2020,DAM0005,POINT (96.848496 18.255561),Bago (East),Kyaukkyi,MMR007011
0,3700.0,NaN,61.0,230,NaN,Thaton,NaN,None,None,310.0,1100.0,NaN,9491.0,220.0,578.0,S,Bilin,280.0,1.00,0.0,131,0.05,NaN,0.0,-,280.0,Local BOT,3138.0,NaN,84.4,Thaton,190.0,HCDG,Bilin,LocMoU,Mon,Bilin,5992.0,2250.0,Bilin,280,9711,None,Bilin River,HP,Myanmar,97.243546,17.519723,Proposed,Belin,2021,DAM0006,POINT (97.24354599999999 17.519723),Mon,Bilin,MMR011010
0,NaN,NaN,NaN,None,NaN,None,NaN,None,None,76.2,600.0,NaN,119.0,602.0,17.0,S,Setotaya,150.0,1.00,0.0,138,0.09,NaN,0.0,0.41,150.0,Sole Investment,1694.0,NaN,34.3,Minbu,79.0,MOALI,Mone Chaung,GOM Plan,Magway,Ayeyarwady,2491.0,2305.0,Mone Chaung,150,721,None,Mon Chaung,MP,Myanmar,94.081800,20.766090,Proposed,Buywa (upper),2019,DAM0007,POINT (94.0818 20.76609),Magway,Sidoktaya,MMR009011
0,NaN,NaN,NaN,None,NaN,None,NaN,None,None,38.1,NaN

In [55]:
new_df.to_csv('data/Hydropower_Coordinates_with_location.csv',sep=',')